In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
%matplotlib inline
# import missingno as mn


df = pd.read_csv('dataset.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 35 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  4424 non-null   int64  
 1   Application mode                                4424 non-null   int64  
 2   Application order                               4424 non-null   int64  
 3   Course                                          4424 non-null   int64  
 4   Daytime/evening attendance                      4424 non-null   int64  
 5   Previous qualification                          4424 non-null   int64  
 6   Nacionality                                     4424 non-null   int64  
 7   Mother's qualification                          4424 non-null   int64  
 8   Father's qualification                          4424 non-null   int64  
 9   Mother's occupation                      

In [7]:
print(df.dtypes)
print('===========')
print(df.info())

Marital status                                      int64
Application mode                                    int64
Application order                                   int64
Course                                              int64
Daytime/evening attendance                          int64
Previous qualification                              int64
Nacionality                                         int64
Mother's qualification                              int64
Father's qualification                              int64
Mother's occupation                                 int64
Father's occupation                                 int64
Displaced                                           int64
Educational special needs                           int64
Debtor                                              int64
Tuition fees up to date                             int64
Gender                                              int64
Scholarship holder                                  int64
Age at enrollm

# Fields to be removed

Marital Status
Application Mode
Previous Qualifications
Nacionality
Mother's qualification
Father's qualification
Mother's occupation
Father's occupation
Educational Special Needs
Age at enrollment
Curricular units 1st sem (evaluations)
Curricular units 1st sem (approved)
Curricular units 1st sem (without evaluations)
Curricular units 2nd sem (evaluations) 
Curricular units 2nd sem (approved)
Curricular units 2nd sem (without evaluations) 
Inflation rate
GDP

In [ ]:
columns_to_remove = [
    'Marital Status', 'Application Mode', 'Previous Qualifications', 'Nacionality',
    'Mother\'s qualification', 'Father\'s qualification', 'Mother\'s occupation', 
    'Father\'s occupation', 'Educational Special Needs', 'Age at enrollment', 
    'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (approved)', 
    'Curricular units 1st sem (without evaluations)', 'Curricular units 2nd sem (evaluations)', 'Curricular units 2nd sem (approved)'
    'Curricular units 2nd sem (without evaluations)', 'Inflation Rate', 'GDP'
]

df.drop(columns=columns_to_remove, inplace=True)